# Case Study: `Strategy`

## 1.1 Classic Strategy

In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity

### The `Context`

In [2]:
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '_total'):
            self._total = sum(item.total() for item in self.cart)
        return self._total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = 'Order total: {:.2f} due: {:.2f}'
        return fmt.format(self.total(), self.due())

### The `Strategy`
An abstract base class

In [3]:
class Promotion(ABC):
    @abstractmethod
    def discount(self, order):
        '''Return discount as a positive dollar amount'''

### `Concrete` strategies

In [4]:
class FidelityPromo(Promotion):
    '''5% discount for customers with 1000 or more fidelity points'''
    
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity > 1000 else 0

In [5]:
class BulkItemPromo(Promotion):
    '''10% discount for each LineItem with 20 or more units'''
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount = item.total() * 0.1
        return discount

In [6]:
class LargeOrderPromo(Promotion):
    '''7% discount for orders with 10 or more distinct items'''
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

### Testing

In [7]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

In [8]:
cart = [LineItem('banana', 4, 0.5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [9]:
Order(joe, cart, FidelityPromo())

Order total: 42.00 due: 42.00

In [10]:
Order(ann, cart, FidelityPromo())

Order total: 42.00 due: 39.90

In [11]:
banana_cart = [LineItem('banana', 30, 0.5),
               LineItem('apple', 10, 1.5)]

In [12]:
Order(joe, banana_cart, BulkItemPromo())

Order total: 30.00 due: 28.50

In [13]:
long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]

In [14]:
Order(joe, long_order, LargeOrderPromo())

Order total: 10.00 due: 9.30

In [15]:
Order(joe, cart, LargeOrderPromo())

Order total: 42.00 due: 42.00

## 1.2 Function-Oriented Strategy

In [16]:
from collections import namedtuple

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.quantity * self.price

### The `Context`

In [17]:
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '_total'):
            self._total = sum(item.total() for item in self.cart)
        return self._total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = 'Order total: {:.2f} due: {:.2f}'
        return fmt.format(self.total(), self.due())

### `Concrete` strategies

In [18]:
def fidelity_promo(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

In [19]:
def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

In [20]:
def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

### Testing

In [21]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

In [22]:
cart = [LineItem('banana', 4, 0.5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [23]:
Order(joe, cart, fidelity_promo)

Order total: 42.00 due: 42.00

In [24]:
Order(ann, cart, fidelity_promo)

Order total: 42.00 due: 39.90

In [25]:
banana_cart = [LineItem('banana', 30, 0.5),
               LineItem('apple', 10, 1.5)]

In [26]:
Order(joe, banana_cart, bulk_item_promo)

Order total: 30.00 due: 28.50

In [27]:
long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]

In [28]:
Order(joe, long_order, large_order_promo)

Order total: 10.00 due: 9.30

In [29]:
Order(joe, cart, large_order_promo)

Order total: 42.00 due: 42.00

### Choosing the best strategy

In [30]:
promos = ['fidelity_promo', 'bulk_item_promo', 'large_order_promo']

def best_promo(order):
    '''Select the best discount availbale'''
    return max(promo(order) for promo in promos)